# Upgrade do artigo original

A ideia aqui é:
* salvar numpy arrays e usar elas diretamente no tensorflow, ao inves de converter em imagem
* diminuir o batch size
* usar um timeframe maior pra pagar menos taxas nas operações

In [3]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
#from zipfile import ZipFile
import zipfile
from timeit import default_timer as timer
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
from keras.utils import load_img
from keras.utils import save_img
from keras.utils import img_to_array
from keras.utils import array_to_img
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.optimizers import Adagrad
from keras.optimizers import SGD
import funcoes as f
import asyncio
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # or any {'0', '1', '2'}
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import plotly.express as px
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from joblib import dump, load
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import absl.logging
from tqdm.auto import tqdm

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

absl.logging.set_verbosity(absl.logging.ERROR)
tf.get_logger().setLevel('WARNING')
#tf.keras.mixed_precision.set_global_policy("mixed_float16")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
# define o ticker do ativo que vou usar pra treinar o modelo
ticker = "BTCUSDT"
# timeframes precisa estar em ordem crescente e começar em 1s ou 1m
timeframes = ("1m", "5m", "15m", "30m", "1h")#, "2h", "4h", "8h", "1d")
# quantos períodos vamos olhar pro passado
lookback = 20
# numero de quantis pra usar no markov transition field. Precisa tunar.
quantis = 3
# pega número de timeframes por minuto ou segundo
timeframes_padronizado = tuple(f.timeframes_mesma_unidade(timeframes))
# tamanho das batches de treinamento
batch_size = 1024#512
# set seed pros resultados não variarem
seed = np.random.randint(99999)
seed = 777
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)
# pega tamanho da imagem (input)
img_shape = f.image_shape()
# porcentagem dos dados que vou usar pra treino e teste
pct_imagens_teste = 0.1
# numero de modelos pro ensemble
n_modelos=5
# proporção pra diminuir as imagens no momento da criação 1 é imagem inteira
img_size = 1.0
# investimento inicial em dolares
investimento_inicial = 1000
# fee em %
fee = 0.1

# Prepara os dados

### Baixa e corrige dados

In [3]:
# baixa os dados e junta cada timeframe e junta em 1 csv por timeframe
# retorna None se arquivo processado já existe e timeframe se ele criou um arquivo do 0
lista_processados = []
for timeframe in timeframes:
    lista_processados.append(f.baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017))

BTCUSDT-1m já processado
BTCUSDT-5m já processado
BTCUSDT-15m já processado
BTCUSDT-30m já processado
BTCUSDT-1h já processado


In [4]:
# insere linhas sem dados e preenche com 0
# só corrige os timeframes que foram processados anteriormente
lista_p_correcao = [item for item in lista_processados if item != None]
f.corrige_arquivos(lista_p_correcao)

Nada corrigido


### Cria janelas, GAF's e salva imagens
Imagens ficam localizadas na pasta dados e são separadas por clissificação de compra ou venda

In [5]:
# cria uma lista de numpy arrays com os dados de cada timeframe.
# Eles estão organizados na mesma ordem que os timeframes
dfs_close = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1][["Close", "Close time"]]).to_numpy() for timeframe in timeframes]
dfs_close = tuple(dfs_close)

# faz a mesma coisa pros dados de volume
dfs_volume = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Volume"])[::-1][["Volume", "Close time"]]).to_numpy() for timeframe in timeframes]
dfs_volume = tuple(dfs_volume)

#dfs_close[0], dfs_volume[0]

In [22]:
# cria arquivo npz vazio
np.savez_compressed(f'./Dados/Arrays/teste/long0/arrays')
np.savez_compressed(f'./Dados/Arrays/teste/short1/arrays')
np.savez_compressed(f'./Dados/Arrays/treino/long0/arrays')
np.savez_compressed(f'./Dados/Arrays/treino/short1/arrays')

def gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
    if decisao == None or (0 in df_janela):
        return
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    # salva como imagem na pasta (treino ou teste)/(long ou short)
    
    #save_img(f'./Dados/Imagens/{pasta}/{decisao}/{timestamp}.png', array_to_img(img_array), scale=False)
    np.save(f'{timestamp}', img_array)
    z = zipfile.ZipFile(f'./Dados/Arrays/{pasta}/{decisao}/arrays.npz', "a",zipfile.ZIP_DEFLATED)
    z.write(f'{timestamp}.npy')
    z.close()
    os.remove(f'{timestamp}.npy')
    
    return

In [32]:
# gerando cada array em um arquivo individual. Fica muito pesado. Tentar diminuir tamanho pra float16 ou menor
def gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, pasta, img_size):
    # tentativa de criar um arquivo npz com todas as numpy arrays. muito lento
    df_janela, decisao, timestamp = f.cria_janela(linha, dfs_close, lookback)
    
    # se decisao não é long nem short ou tem um buraco nos dados no meio da janela, pula a iteração
    if decisao == None or (0 in df_janela):
        return
    
    df_janela_volume = f.cria_janela(linha, dfs_volume, lookback)[0]
    
    # cria gaf images
    img_array = np.vstack((f.cria_gaf(df_janela.T, quantis, img_size), f.cria_gaf(df_janela_volume.T, quantis, img_size)))
    
    # salva como imagem na pasta (treino ou teste)/(long ou short)

    np.save(f'./Dados/Arrays/{pasta}/{decisao}/{timestamp}', np.float16(img_array))
    
    return

In [33]:


if __name__ == '__main__':
    ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])
    print("qtd de imagens para treino", ultima_linha-int(ultima_linha*pct_imagens_teste))
    print("qtd de imagens para teste", int(ultima_linha*pct_imagens_teste))
    # linhas de teste são as primeiras x%, pois os dados mais recentes vem antes
    # linhas de treino são as restantes
    linhas_teste = range(1, int(ultima_linha*pct_imagens_teste))
    # linhas_validacao = 
    linhas_treino = range(int(ultima_linha*pct_imagens_teste)+1, ultima_linha+1)
    
    for linha in tqdm(linhas_teste):
        gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, "teste", img_size)
    #for linha in tqdm(linhas_treino):
        #gera_numpy_arrays(linha, dfs_close, dfs_volume, lookback, quantis, "treino", img_size)
    
    # multithreading está quebrado!
    # cria imagens usando multithreading
    #f.roda_async2(linhas_treino, linhas_teste, dfs_close, dfs_volume, lookback, quantis)
    # testar sem escalar as imagens no untitled.ipynb antes

qtd de imagens para treino 2542752
qtd de imagens para teste 282528


  0%|          | 0/282527 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [26]:
x = np.load(f'./Dados/Arrays/teste/long0/1669998239999.npy')

In [28]:
x

array([[[-0.52850115,  0.        ,  0.5714286 ],
        [-0.7841054 ,  0.33765015,  0.5714286 ],
        [-0.9049852 ,  0.54342425,  0.42857143],
        ...,
        [-0.95129716,  0.35246924,  0.33333334],
        [-0.62483436, -0.22046293,  0.5       ],
        [-0.5164806 , -0.34709576,  0.5       ]],

       [[-0.7841054 , -0.33765015,  0.5714286 ],
        [-0.9476112 ,  0.        ,  0.5714286 ],
        [-0.99548787,  0.22806668,  0.42857143],
        ...,
        [-0.7044524 ,  0.7419498 ,  0.16666667],
        [-0.19784003,  0.2507708 ,  0.5       ],
        [-0.06692035,  0.12099573,  0.5       ]],

       [[-0.9049852 , -0.54342425,  0.33333334],
        [-0.99548787, -0.22806668,  0.33333334],
        [-0.9908933 ,  0.        ,  0.16666667],
        ...,
        [-0.02341227,  0.9997259 ,  0.16666667],
        [ 0.53040904,  0.84774184,  0.5       ],
        [ 0.63751346,  0.7704392 ,  0.5       ]],

       ...,

       [[ 0.04202244, -0.07076511,  0.2857143 ],
        [-0

In [ ]:
# Ver de fazer o pre processamento direto ao inves de salvar no disco as numpy arrays
# assim da pra usar a precisão máxima, não ocupa espaço em disco e pode ser até mais rápido

# Prova que o cria_janelas funciona perfeitamente

In [9]:
if __name__ == '__main__':
    # criação de DF gigante e cheio de nan's

    def une_timeframes(timeframes=timeframes):
        # junta todos os timeframes em um df só
        # esse é o jeito lento e serve apenas pra fazer uma prova real
        # pega o primeiro timeframe com as colunas desejadas
        df = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframes[0]}.csv", index_col = "Close time"
                         , usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
        #df.index = pd.to_datetime(df.index, unit="ms")

        # coloca o sufixo nas colunas do menor timeframe
        df.columns = df.columns+f"_{timeframes[0]}"
        print(timeframes[0])

        # junta todos os outros timeframes e coloca o sufixo correto
        for timeframe in timeframes[1:]:
            print(timeframe)
            df2 = pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", index_col = "Close time",
                             usecols=["Close time","Open", "High", "Low", "Close", "Volume", "Number of trades"])
            df2.columns = df2.columns+f"_{timeframe}"
            df = df.merge(df2, how="left", on=["Close time"])#, suffixes=(None, f"_{timeframe}"))


        # transforma o index em datetime
        df.index = pd.to_datetime(df.index, unit="ms")

        #inverte a ordem do df pq vamos usar ele do fim pro começo pra fazer as janelas
        df.sort_index(axis=0, inplace = True, ascending=False)
        return(df)
    df = une_timeframes(timeframes)
    df[99:].head(20)

1m
5m
15m
30m
1h


In [10]:
df[99:].head(20)

,Open_1m,High_1m,Low_1m,Close_1m,Volume_1m,Number of trades_1m,Open_5m,High_5m,Low_5m,Close_5m,...,Low_30m,Close_30m,Volume_30m,Number of trades_30m,Open_1h,High_1h,Low_1h,Close_1h,Volume_1h,Number of trades_1h
Close time,,,,,,,,,,,,,,,,,,,,,
2022-12-31 22:20:59.999,16531.61,16537.39,16528.91,16532.26,110.79296,2708,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:19:59.999,16519.14,16531.70,16506.36,16531.62,638.64815,8301,16564.95,16565.62,16506.36,16531.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:18:59.999,16547.34,16547.85,16515.16,16519.14,408.04045,6108,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:17:59.999,16554.12,16554.23,16545.78,16547.34,96.98766,2665,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:16:59.999,16562.27,16563.55,16553.57,16554.12,109.60176,2619,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:15:59.999,16564.95,16565.62,16561.78,16562.73,76.53378,1809,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:14:59.999,16562.26,16567.49,16562.14,16565.38,98.09439,2583,16556.18,16567.49,16555.75,16565.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:13:59.999,16562.15,16563.80,16560.68,16562.26,70.03443,1721,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 22:12:59.999,16565.11,16565.16,16560.26,16561.81,77.08899,1988,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
colunas = [coluna for coluna in df.columns if "Close" in coluna]
df[colunas]

,Close_1m,Close_5m,Close_15m,Close_30m,Close_1h
0,16542.40,16542.40,16542.4,16542.4,16542.4
1,16536.42,NaN,NaN,NaN,NaN
2,16537.78,NaN,NaN,NaN,NaN
3,16540.52,NaN,NaN,NaN,NaN
4,16539.28,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2826475,4261.48,4261.48,NaN,NaN,NaN
2826476,4261.48,NaN,NaN,NaN,NaN
2826477,4280.56,NaN,NaN,NaN,NaN
2826478,4261.48,NaN,NaN,NaN,NaN


In [28]:
linha = 99
df2 = pd.DataFrame()
for coluna in colunas:
    df2[coluna] = df[coluna].iloc[linha:].dropna().head(lookback).reset_index(drop=True)
df2

,Close_1m,Close_5m,Close_15m,Close_30m,Close_1h
0,16532.26,16531.62,16565.38,16548.28,16548.28
1,16531.62,16565.38,16548.28,16563.84,16568.60
2,16519.14,16556.18,16560.53,16568.60,16570.14
3,16547.34,16543.94,16563.84,16570.71,16577.79
4,16554.12,16548.28,16566.31,16570.14,16582.08
5,16562.73,16549.59,16568.60,16572.17,16600.39
6,16565.38,16556.15,16570.46,16577.79,16590.10
7,16562.26,16560.53,16570.71,16577.13,16586.43
8,16561.81,16560.07,16569.76,16582.08,16600.89
9,16565.11,16557.92,16570.14,16583.43,16575.44


In [29]:
pd.DataFrame(f.cria_janela(linha, dfs_close, lookback)[0])

,0,1,2,3,4
0,16532.26,16531.62,16565.38,16548.28,16548.28
1,16531.62,16565.38,16548.28,16563.84,16568.60
2,16519.14,16556.18,16560.53,16568.60,16570.14
3,16547.34,16543.94,16563.84,16570.71,16577.79
4,16554.12,16548.28,16566.31,16570.14,16582.08
5,16562.73,16549.59,16568.60,16572.17,16600.39
6,16565.38,16556.15,16570.46,16577.79,16590.10
7,16562.26,16560.53,16570.71,16577.13,16586.43
8,16561.81,16560.07,16569.76,16582.08,16600.89
9,16565.11,16557.92,16570.14,16583.43,16575.44


In [31]:
# prova que o timestamp também está certo
f.cria_janela(linha, dfs_close, lookback)[2]

1672525259999

In [32]:
pd.to_datetime(f.cria_janela(linha, dfs_close, lookback)[2], unit="ms")

Timestamp('2022-12-31 22:20:59.999000')